In [54]:
pip install pytz


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 14.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.87.0
    Uninstalling google-api-python-client-2.87.0:
      Successfully uninstalled google-api-python-client-2.87.0

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
pip install backports.zoneinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for backports.zoneinfo (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      /opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/setuptools/config/setupcfg.py:516: SetuptoolsDeprecationWarning: The license_file parameter is deprecated, use license_files instead.
        warnings.warn(msg, warning_class)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-13-arm64-cpython-311
      creating build/lib.macosx-13-arm64-cpython-311/backports
      copying src/backports/__init__.py -> build/lib.macosx-13-arm64-cpython-311/backports
      creating b

In [77]:
from __future__ import print_function

import datetime
import os.path
import pytz

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from zoneinfo import ZoneInfo


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


def getEvents():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('calendar_token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                './client_secret_241823786615-5oh4q20ihegr14g6bpst89ke47v65ikf.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('calendar_token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        print('Getting the upcoming 10 events')
        events_result = service.events().list(calendarId='primary', timeMin=now,
                                              maxResults=10, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

        # Prints the start and name of the next 10 events
    
    except HttpError as error:
        print('An error occurred: %s' % error)
    return events


def checkAvailability(start_time, end_time):
        events = getEvents()

        for event in events:
        # Define the start and end times of your desired interval
        #Example one: non overlapping
            
            # start_time = datetime.time(hour=2, minute=0, second=0, tzinfo= ZoneInfo('US/Eastern'))
            start_date= datetime.datetime.combine(datetime.datetime.today(),start_time )
            # end_time = datetime.time(hour=3, minute=0, second=0,tzinfo= ZoneInfo('US/Eastern'))
            end_date= datetime.datetime.combine(datetime.datetime.today(),end_time )

            def parseDate(rawDate):
                #Transform the datetime given by the API to a python datetime object.
                return datetime.datetime.strptime(rawDate[:-6]+ rawDate[-6:].replace(":",""), '%Y-%m-%dT%H:%M:%S%z')

            eventStart = parseDate(event['start'].get('dateTime', event['start'].get('date'))) 
            eventEnd = parseDate(event['end'].get('dateTime', event['end'].get('date')))
            
                # Check for an overlap between the event and the desired interval
            if eventStart < end_date and eventEnd> start_date:
                print('The desired interval overlaps with event:', event['summary'])
                return
            
        print('I can')
        # add to calender
         
    

start_time = datetime.time(hour=2, minute=0, second=0, tzinfo= ZoneInfo('US/Eastern'))
end_time = datetime.time(hour=3, minute=0, second=0,tzinfo= ZoneInfo('US/Eastern'))
checkAvailability(start_time, end_time)

Getting the upcoming 10 events
I can
